In [1]:
import numpy as np
import pandas as pd
import json,re,csv, requests
from bs4 import BeautifulSoup

In [2]:
genres = {}

# Rap

In [3]:
# Ranker (Rap)
url = 'https://www.ranker.com/crowdranked-list/the-greatest-rappers-of-all-time'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

rappers_ranker_1 = []
for artist in soup.findAll('meta', {'itemprop':'name'}):
    if artist['content'] != 'The Greatest Rappers of All Time':
        rappers_ranker_1.append(artist['content'].lower().strip())

In [4]:
# Ranker 2 (Rap)
url = 'https://www.ranker.com/list/best-hip-hop-artists-in-2018/ranker-music'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

rappers_ranker_2 = []
for artist in soup.findAll('meta', {'itemprop':'name'}):
    if artist['content'] != 'The Best Rappers Of 2018, Ranked':
        rappers_ranker_2.append(artist['content'].lower().strip())

In [5]:
genres['rap'] = list(set(sorted(rappers_ranker_1 + rappers_ranker_2)))

# Metal

In [6]:
url = 'https://www.thetoptens.com/top-heavy-metal-bands/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

# Find all commenters
commenters = []
for commenter in soup.findAll('p'):
    commenter_username = commenter.find('b')
    if commenter_username:
        commenters.append(commenter_username.get_text().lower().strip())

# Toptens (Metal)
metal_toptens = []
for artist in soup.findAll('b')[1:]:
    artist_name = artist.get_text().lower().strip()
    if artist_name not in commenters:
        if artist_name[0:3] != '1. ':
            metal_toptens.append(artist_name)

In [7]:
genres['metal'] = list(set(sorted(metal_toptens)))

# Country

In [8]:
# Ranker (Country)
url = 'https://www.ranker.com/list/top-country-artists-of-all-time/samantha-dillinger'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

country_ranker = []
for artist in soup.findAll('meta', {'itemprop':'name'}):
    if artist['content'] != 'The Top Country Artists of All Time':
        country_ranker.append(artist['content'].lower().strip())

In [9]:
# Ranker (Modern country)
url = 'https://www.ranker.com/list/best-new-country-artists/ranker-music'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

modern_country_ranker = []
for artist in soup.findAll('meta', {'itemprop':'name'}):
    if artist['content'] != 'The Best New Country Artists':
        modern_country_ranker.append(artist['content'].lower().strip())

In [10]:
genres['country'] = list(set(sorted(country_ranker + modern_country_ranker)))

# Rock

In [11]:
# Ranker (Rock)
url = 'https://www.ranker.com/crowdranked-list/the-best-rock-bands-of-all-time'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

rock_ranker = []
for artist in soup.findAll('meta', {'itemprop':'name'}):
    if artist['content'] != 'The Best Rock Bands of All Time':
        rock_ranker.append(artist['content'].lower().strip())

In [12]:
# Ranker (Modern Rock)
url = 'https://www.ranker.com/list/modern-rock-bands-and-musicians/reference'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

modern_rock_ranker = []
for artist in soup.findAll('meta', {'itemprop':'name'}):
    if artist['content'] != 'The Best Modern Rock Bands/Artists':
        modern_rock_ranker.append(artist['content'].lower().strip())

In [13]:
# Ranker (Indie Rock)
url = 'https://www.ranker.com/list/indie-bands-and-artists/reference'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

indie_ranker = []
for artist in soup.findAll('meta', {'itemprop':'name'}):
    if artist['content'] != 'The Best Indie Bands & Artists':
        indie_ranker.append(artist['content'].lower().strip())

In [14]:
genres['rock'] = list(set(sorted(rock_ranker + modern_rock_ranker + indie_ranker)))

# Pop

In [15]:
# Billboard (Pop)
url = 'https://www.billboard.com/charts/greatest-of-all-time-pop-songs-artists'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

pop_billboard = []
for artist in soup.findAll('a', {'href':re.compile('/music/')}):
    artist = artist.get_text().replace('\n', '')
    pop_billboard.append(artist.lower().strip())

In [16]:
genres['pop'] = list(set(sorted(pop_billboard)))

# Soul

In [17]:
# Ranker (Soul)
url = 'https://www.ranker.com/list/soul-music-bands-and-musicians/reference'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

soul_ranker = []
for artist in soup.findAll('meta', {'itemprop':'name'}):
    if artist['content'] != 'The Best Soul Singers/Groups of All Time':
        soul_ranker.append(artist['content'].lower().strip())

In [18]:
genres['soul'] = list(set(sorted(soul_ranker)))

# Remove duplicates

In [20]:
# Country & pop (becomes pop)
genres['country'] = sorted([artist for artist in genres['country'] if artist not in genres['pop']])

# Country & rock (becomes rock)
genres['country'] = sorted([artist for artist in genres['country'] if artist not in genres['rock']])

# Metal & rock (becomes metal)
genres['rock'] = sorted([artist for artist in genres['rock'] if artist not in genres['metal']])

# Pop & rock (becomes pop)
genres['rock'] = sorted([artist for artist in genres['rock'] if artist not in genres['pop']])

# Pop & rap (becomes rap)
genres['pop'] = sorted([artist for artist in genres['pop'] if artist not in genres['rap']])

# Convert genres dictionary to json

In [24]:
json_genres = json.dumps(genres, indent=4, sort_keys=True)
file = open('D:/Anaconda3/Scripts/7 實驗進度/DALI/example/dataset/json_genres.json', 'w')
file.write(json_genres)
file.close()

# lyrics_classifier

In [38]:
# Import dependencies
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import EnglishStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

import json, re, string, warnings, nltk
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier

In [29]:
# Read lyrics data
lyrics = pd.read_csv('D:/Anaconda3/Scripts/7 實驗進度/DALI/example/dataset/lyrics.csv', encoding='unicode_escape')
lyrics['artist_name'] = lyrics.apply(lambda x: x['artist_name'].lower(), axis=1)
lyrics['original_line'] = lyrics['line']

# Remove misclassified artists

In [30]:
# Import genre artist lists
file = open('D:/Anaconda3/Scripts/7 實驗進度/DALI/example/dataset/json_genres.json')
genres = json.load(file)
file.close()

In [31]:
# Check whether artists in a given genre are (potentially) misclassified
def artistCheck(genre):
    misclasses = []
    artists = lyrics[lyrics.genre==genre].artist_name.unique()
    for artist in artists:
        if artist not in genres[genre]:
            misclasses.append(artist)
    return(misclasses)

In [32]:
# Collect misclassified artists of every genre
misclass_dict = {}
for genre in lyrics.genre.unique():
    misclass_dict[genre] = artistCheck(genre)
misclass_dict

{'country': ['kalie shorr',
  'shania twain',
  'dustin lynch',
  'faith hill',
  'john denver',
  'tyler, the creator',
  'keith whitley',
  'aaron watson',
  'maggie rose',
  'toby keith',
  'jerry reed',
  'carrie underwood',
  'jillian jacqueline',
  'delta rae',
  'tim mcgraw',
  'the oak ridge boys',
  'taylor acorn',
  'george jones & tammy wynette',
  'blake shelton',
  'ben gallaher',
  'hank williams jr.'],
 'metal': ['queensrÿche',
  'trivium',
  'a$ap rocky',
  'killswitch engage',
  'godsmack',
  'testament',
  'dokken',
  'korn',
  'motörhead',
  'tool',
  "guns n' roses"],
 'pop': ['andrea bocelli & céline dion', 'drake'],
 'rap': ['lauryn hill',
  'ghostface killah',
  'big l',
  'tyler, the creator',
  'a$ap rocky',
  'trippie redd',
  'tech n9ne',
  'rich the kid',
  'nate dogg',
  'll cool j',
  'dmx',
  '2 chainz',
  'mac miller',
  'slick rick',
  'jay-z',
  'method man',
  'royce da 5\x929\x94',
  'ludacris',
  't.i.',
  'wiz khalifa',
  'j.i.d',
  'tee grizzley',

In [33]:
# Remove all actual misclasses
lyrics = lyrics[~((lyrics.genre=='country') & (lyrics.artist_name=='tyler, the creator'))]
lyrics = lyrics[~((lyrics.genre=='metal') & (lyrics.artist_name=='a$ap rocky'))]
lyrics = lyrics[~((lyrics.genre=='pop') & (lyrics.artist_name=='drake'))]
lyrics = lyrics[~((lyrics.genre=='rock') & (lyrics.artist_name=='hieroglyphics'))]
lyrics = lyrics[~((lyrics.genre=='rock') & (lyrics.artist_name=='wild belle'))]
lyrics = lyrics[~((lyrics.genre=='soul') & (lyrics.artist_name=='prince ea'))]
lyrics = lyrics[~((lyrics.genre=='soul') & (lyrics.artist_name=='sam smith'))]
lyrics = lyrics[~((lyrics.genre=='soul') & (lyrics.artist_name=='sir sly'))]
lyrics = lyrics[~((lyrics.genre=='soul') & (lyrics.artist_name=='telly'))]
lyrics = lyrics.reset_index(drop=True)

In [34]:
# Remove artists that "blur the line" b/t genres [manual decision]
removes_country = []
removes_metal = []
removes_pop = ['chris brown', 'nickelback', 'sheryl crow', 'taylor swift', 'matchbox twenty']
removes_rap = []
removes_rock = []
removes_soul = []
removes = removes_country + removes_metal + removes_pop + removes_rap + removes_rock + removes_soul

lyrics = lyrics[~lyrics.artist_name.isin(removes)]

# Text preprocessing

In [35]:
# Process text for classification modeling
def preprocessText(text, remove_stops=False):
    
    # Remove everything between hard brackets
    text = re.sub(pattern="\[.+?\]( )?", repl='', string=text)

    # Change "walkin'" to "walking", for example
    text = re.sub(pattern="n\\\' ", repl='ng ', string=text)

    # Remove x4 and (x4), for example
    text = re.sub(pattern="(\()?x\d+(\))?", repl=' ', string=text)

    # Fix apostrophe issues
    text = re.sub(pattern="\\x91", repl="'", string=text)
    text = re.sub(pattern="\\x92", repl="'", string=text)
    text = re.sub(pattern="<u\+0092>", repl="'", string=text)
    
    # Make lowercase
    text = text.lower()
    
    # Special cases/words
    text = re.sub(pattern="'til", repl="til", string=text)
    text = re.sub(pattern="'til", repl="til", string=text)
    text = re.sub(pattern="gon'", repl="gon", string=text)

    # Remove \n from beginning
    text = re.sub(pattern='^\n', repl='', string=text)

    # Strip , ! ?, : and remaining \n from lyrics
    text = ''.join([char.strip(",!?:") for char in text])
    text = text.replace('\n', ' ')

    # Remove contractions
    # specific
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"won\’t", "will not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"can\’t", "can not", text)
    text = re.sub(r"let's", "let us", text)
    text = re.sub(r"let\’s", "let us", text)
    text = re.sub(r"ain't", "aint", text)
    text = re.sub(r"ain\’t", "aint", text)

    # general
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"n\’t", " not", text)
    text = re.sub(r"\’re", " are", text)
    text = re.sub(r"\’s", " is", text)
    text = re.sub(r"\’d", " would", text)
    text = re.sub(r"\’ll", " will", text)
    text = re.sub(r"\’t", " not", text)
    text = re.sub(r"\’ve", " have", text)
    text = re.sub(r"\’m", " am", text)

    # Remove remaining punctuation
    punc = string.punctuation
    text = ''.join([char for char in text if char not in punc])

    # Remove stopwords
    if remove_stops:
        stops = stopwords.words('english')
        text = ' '.join([word for word in text.split(' ') if word not in stops])
    
    # Remove double spaces and beginning/trailing whitespace
    text = re.sub(pattern='( ){2,}', repl=' ', string=text)
    text = text.strip()
    
    return(text)

In [36]:
lyrics['line'] = lyrics.apply(lambda x: preprocessText(x['line']), axis=1)

In [41]:
lyrics

,line,song_id,song_name,artist_id,artist_name,characters,genre,original_line,line_stems
0,i poured your johnnie walker blue right down t...,2429721,I Need You,643585,kalie shorr,1881,country,I poured your Johnnie Walker Blue right down t...,i pour your johnni walker blue right down the ...
1,his mama thinks he is an angel follows the sav...,2429715,God Sees Everything,643585,kalie shorr,1287,country,His mama thinks hes an angel\nFollows the Sav...,his mama think he is an angel follow the savio...
2,i am just trying to catch a buzz minding my ow...,2429717,Throwin' Shade,643585,kalie shorr,1305,country,I'm just trying to catch a buzz\nMindin' my ow...,i am just tri to catch a buzz mind my own litt...
3,sun is always gonna set in the west lipsticks ...,3019461,Nothin New,643585,kalie shorr,1892,country,[Verse 1]\nSun's always gonna set in the west\...,sun is alway gonna set in the west lipstick an...
4,kill the engine turn the headlights off drive ...,2429718,Young & Restless,643585,kalie shorr,2046,country,"Kill the engine, turn the headlights off\nDriv...",kill the engin turn the headlight off drive aw...
...,...,...,...,...,...,...,...,...,...
8862,got myself an angel straight from the heavens ...,1618057,I Got Heaven Right Here on Earth,1664,the temptations,1788,soul,Got myself an angel\nStraight from the heavens...,got myself an angel straight from the heaven a...
8863,baby our love was strong yesterday now you wat...,3832684,Why Did You Leave Me Darling,1664,the temptations,1063,soul,"Baby, our love was strong yesterday\n(Now you ...",babi our love was strong yesterday now you wat...
8864,your eyes tell a story such a sad story of the...,3129527,Ain't No Justice,1664,the temptations,1098,soul,"Your eyes tell a story, such a sad story\nOf t...",your eye tell a stori such a sad stori of the ...
8865,guess it is true i am not good at a onenight s...,3705373,Stay With Me,1664,the temptations,1167,soul,"[Verse 1]\nGuess it's true, I'm not good at a ...",guess it is true i am not good at a onenight s...


# Stem words

In [42]:
stemmer = EnglishStemmer()
lyrics['line_stems'] = lyrics.apply(lambda x: ' '.join([stemmer.stem(word) for word in x['line'].split(' ')]), axis=1)
#lyrics['unique_stems'] = lyrics.apply(lambda x: ' '.join([stemmer.stem(word) for word in x['unique_words']]), axis=1)

# Genre Classifier Models

In [43]:
# Combine metal & rock
lyrics['genre'] = lyrics.apply(lambda x: 'metal/rock' if x['genre'] in ['metal', 'rock'] else x['genre'], axis=1)

# Train/test split
train, test = train_test_split(lyrics, test_size=0.2, stratify=lyrics.genre, random_state=1)

# Classifier - Naive Bayes

In [44]:
# Naive Bayes model w/ k-fold CV
text_mnb = Pipeline([('vect', CountVectorizer()),
                     ('mnb', MultinomialNB(fit_prior=False))])
text_mnb = text_mnb.fit(train.line_stems, train.genre)
cross_val_score(estimator=text_mnb, X=train.line_stems, y=train.genre, cv=7).mean()

0.7063501162374193

In [45]:
# Naive Bayes test results
print(text_mnb.score(y=test.genre, X=test.line_stems))
preds = text_mnb.predict(test.line_stems)
print(classification_report(y_pred=preds, y_true=test.genre))
pd.crosstab(preds, test.genre)

0.7082847141190198
              precision    recall  f1-score   support

     country       0.60      0.66      0.63       296
  metal/rock       0.77      0.72      0.75       595
         pop       0.55      0.57      0.56       231
         rap       0.88      0.91      0.90       303
        soul       0.64      0.63      0.64       289

    accuracy                           0.71      1714
   macro avg       0.69      0.70      0.69      1714
weighted avg       0.71      0.71      0.71      1714



genre,country,metal/rock,pop,rap,soul
row_0,,,,,
country,195,60,23,7,38
metal/rock,51,427,34,7,32
pop,20,44,132,11,33
rap,4,15,14,277,3
soul,26,49,28,1,183


# Classifier - SVM

In [47]:
# SVM model with k-fold CV
text_svm = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2))), ('svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=123))])
text_svm = text_svm.fit(train.line_stems, train.genre)
cross_val_score(estimator=text_svm, X=train.line, y=train.genre, cv=7).mean()

0.730703410544086

In [48]:
# SVM test results
print(text_svm.score(y=test.genre, X=test.line_stems))
preds_svm = text_svm.predict(test.line_stems)
print(classification_report(y_pred=preds_svm, y_true=test.genre))
pd.crosstab(preds_svm, test.genre)

0.7409568261376897
              precision    recall  f1-score   support

     country       0.67      0.68      0.68       296
  metal/rock       0.74      0.82      0.77       595
         pop       0.70      0.48      0.57       231
         rap       0.91      0.94      0.93       303
        soul       0.65      0.64      0.65       289

    accuracy                           0.74      1714
   macro avg       0.74      0.71      0.72      1714
weighted avg       0.74      0.74      0.74      1714



genre,country,metal/rock,pop,rap,soul
row_0,,,,,
country,201,41,14,3,39
metal/rock,58,486,57,8,51
pop,8,23,111,4,13
rap,3,9,15,286,0
soul,26,36,34,2,186


In [49]:
# Which songs did the svm model misclassify?
test['preds_svm'] = preds_svm
misclasses_svm = test[test.preds_svm != test.genre]
misclasses_svm['misclass_combo'] = misclasses_svm.apply(lambda x: x['genre']+'-'+x['preds_svm'], axis=1)

In [50]:
misclasses_svm.misclass_combo.value_counts()

country-metal/rock    58
pop-metal/rock        57
soul-metal/rock       51
metal/rock-country    41
soul-country          39
metal/rock-soul       36
pop-soul              34
country-soul          26
metal/rock-pop        23
pop-rap               15
pop-country           14
soul-pop              13
metal/rock-rap         9
rap-metal/rock         8
country-pop            8
rap-pop                4
country-rap            3
rap-country            3
rap-soul               2
Name: misclass_combo, dtype: int64

In [51]:
misclasses_svm[misclasses_svm.misclass_combo=='country-rap']

,line,song_id,song_name,artist_id,artist_name,characters,genre,original_line,line_stems,preds_svm,misclass_combo
690,i should have known by the way you passed me b...,144193,Undo It,21014,carrie underwood,1962,country,[Verse 1]\nI should've known by the way you pa...,i should have known by the way you pass me by ...,rap,country-rap
656,splish splash i was taking a bath long about a...,1728080,Splish Splash,223639,conway twitty,1274,country,"Splish Splash, I was takin' a bath\nLong about...",splish splash i was take a bath long about a s...,rap,country-rap
1411,gimme some straight talk straight talk and hol...,208250,Straight Talk,20955,dolly parton,1859,country,"[Chorus]\nGimme some straight talk, straight t...",gimm some straight talk straight talk and hold...,rap,country-rap


In [53]:
# Re-train model on full data set
full_text_svm = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2))), ('svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=123))])
full_text_svm = full_text_svm.fit(lyrics.line, lyrics.genre)

# Classifier - XGBoost

In [59]:
# XGB model
vect = TfidfVectorizer(ngram_range=(1,2))
vect.fit_transform(train.line_stems)
vect_train = vect.transform(pd.Series(train.line_stems))
vect_test = vect.transform(pd.Series(test.line_stems))

In [61]:
xgb = XGBClassifier(learning_rate=0.25, subsample=0.8, gamma=1, random_state=123, max_depth=4, max_delta_step=1).fit(vect_train, train.genre)

In [62]:
print(xgb.score(y=test.genre, X=vect_test))
preds_xgb = xgb.predict(vect_test)
print(classification_report(y_pred=preds_xgb, y_true=test.genre))
pd.crosstab(xgb.predict(vect_test), test.genre)

0.7532088681446908
              precision    recall  f1-score   support

     country       0.72      0.62      0.67       296
  metal/rock       0.71      0.86      0.78       595
         pop       0.74      0.53      0.62       231
         rap       0.96      0.95      0.96       303
        soul       0.68      0.64      0.66       289

    accuracy                           0.75      1714
   macro avg       0.76      0.72      0.74      1714
weighted avg       0.76      0.75      0.75      1714



genre,country,metal/rock,pop,rap,soul
row_0,,,,,
country,184,31,11,5,25
metal/rock,85,512,63,4,59
pop,6,11,122,6,19
rap,0,1,10,287,0
soul,21,40,25,1,186


In [63]:
# Which songs did the xgb model misclassify?
test['preds_xgb'] = preds_xgb
misclasses_xgb = test[test.preds_xgb != test.genre]
misclasses_xgb['misclass_combo'] = misclasses_xgb.apply(lambda x: x['genre']+'-'+x['preds_xgb'], axis=1)

In [64]:
misclasses_xgb.misclass_combo.value_counts()

country-metal/rock    85
pop-metal/rock        63
soul-metal/rock       59
metal/rock-soul       40
metal/rock-country    31
soul-country          25
pop-soul              25
country-soul          21
soul-pop              19
pop-country           11
metal/rock-pop        11
pop-rap               10
rap-pop                6
country-pop            6
rap-country            5
rap-metal/rock         4
rap-soul               1
metal/rock-rap         1
Name: misclass_combo, dtype: int64

In [65]:
misclasses_xgb[misclasses_xgb.misclass_combo=='metal/rock-rap'].sort_values('artist_name')

,line,song_id,song_name,artist_id,artist_name,characters,genre,original_line,line_stems,preds_svm,preds_xgb,misclass_combo
7108,this time i came to get mine i saw this cat ru...,338348,Liquid Diet,424,papa roach,1689,metal/rock,"[Chorus 1]\nThis time, I came to get mine\nI s...",this time i came to get mine i saw this cat ru...,rap,rap,metal/rock-rap


# Predict new text - SVM

In [66]:
# Process and predict genre of text (SVM)
def svmLyricClassifier(text):
    text = preprocessText(text)
    text = ' '.join([stemmer.stem(word) for word in text.split(' ')])
    print(full_text_svm.predict([text]))

In [67]:
# Zac Brown Band - Chicken Fried
svmLyricClassifier(
'''
Well I was raised up beneath the shade of a Georgia Pine
And that's home you know
Sweet tea, pecan pie, and homemade wine where the peaches grow
And my house it's not much to talk about
But it's filled with love that's grown in southern ground
''')

['country']


In [68]:
# Nickelback - Photograph
svmLyricClassifier(
'''
Look at this photograph
Every time I do, it makes me laugh
How did our eyes get so red?
And what the hell is on Joey's head?
And this is where I grew up
I think the present owner fixed it up
I never knew we ever went without
The second floor is hard for sneaking out
''')

['metal/rock']


In [69]:
# One Direction - What Makes You Beautiful
svmLyricClassifier(
'''
Baby, you light up my world like nobody else
The way that you flip your hair gets me overwhelmed
But when you smile at the ground, it ain't hard to tell
You don't know, oh, oh, you don't know you're beautiful
If only you saw what I can see
You'll understand why I want you so desperately
Right now I'm looking at you and I can't believe
You don't know, you don't know you're beautiful
That's what makes you beautiful
''')

['pop']


In [70]:
# Wu-Tang Clan - C.R.E.A.M.
svmLyricClassifier(
'''
I grew up on the crime side, the New York Times side
Stayin' alive was no jive
Had secondhands, Mom's bounced on old man
So then we moved to Shaolin land
A young youth, yo, rockin' the gold tooth, 'Lo goose
Only way I be gettin' the G off was drug loot
And let's start it like this, son
Rollin' with this one and that one
Pullin' out gats for fun
''')

['rap']


In [71]:
# Patti LaBelle - My Friend
svmLyricClassifier(
'''
The thought of you helps me carry on
When I feel all hope is gone
I see the world wit brand new eyes
Your love has made me realize
My future looks bright to me
Oh because you are my friend
I've been looking around and you were here all the time
I've been around and around and around and around
I've been looking around and you were here all the time
''')

['soul']


# Predict new text - XGB

In [72]:
# Process and predict genre of text (XGB)
def xgbLyricClassifier(text):
    text = preprocessText(text)
    text = ' '.join([stemmer.stem(word) for word in text.split(' ')])
    text = vect.transform(pd.Series(text))
    return(xgb.predict(text))

In [73]:
# Trippie Redd - Poles1469
xgbLyricClassifier('''
Scum Gang!
I'm toting big shit (big shit)
Get your wig split (wig split)
4-5, I call that the big bitch (big stick)
We don't miss shit (miss shit)
Get your ribs hit (limbs hit)
Hollow tip, hit you with a cross hit (yeaaah)
Big drum on the gun, that's a pump with a red beam
Catch him in the bed while he sleep, that's a wet dream
(That's a wet dream)
Close range headshots, fuck where the vest be
Put a hole in his head, he a dolphin (you a dolphin)
Pull up to the block, you don’t want it (that's that dummy shit)
Pull up with the Glock, you won't want it (brrrah!, stoopid)
Scum Gang, bitch, we on it, you don't want it (ooh, ooh)
1400, bitch, you know I tote a pole
With 6ix9ine, bitch, you know we tote them poles
Tote a pole, tote a pole, tote a pole
You talk down, we gon' let that shit go (grrrat!)
Let it blow, let it blow, let it blow (let that bitch go)
Got the scope, got the scope, got the scope
.223's and I put 'em to your nose (to your nose)
Let it go, bitch, we let that shit go
''')

array(['metal/rock'], dtype=object)

# Predict new text - Naive Bayes

In [74]:
def nbLyricClassifier(text):
    text = preprocessText(text)
    text = ' '.join([stemmer.stem(word) for word in text.split(' ')])
    print(text_mnb.predict([text]))

In [75]:
# Barbra Streisand - Woman In Love
nbLyricClassifier(
'''
life is a moment in space 
when the dream is gone 
it's a lonelier place 
i kiss the morning goodbye 
butdown inside 
you know we never know why 
the road is narrow and long 
when eyes meet eyes 
and the feeling is strong 
i turn away from the wall 
i stumble and fall 
but i give you it all 
i am a woman in love 
and i'd do anything 
to get you into my world 
and hold you within 
it's aright i defend 
overand overagain 
what do i do 
with you eternally mine 
in love there is 
nomeasure of time 
they planned it all at the start 
thatyou and i 
live in each other's heart 
we maybe oceans away 
you feel my love 
i hear what you say 
no truth is ever a lie 
i stumble and fall 
but i give you it all 
i am a woman in love 
and i'd do anything 
to get you into my world 
and hold you within 
it's aright i defend 
overand overagain 
what do i do 
i am a woman in love 
and i'm talkin' to you 
youiknow how you feel 
what a woman can do 
it's a right i defend 
overand overagain 
i am a woman in love 
and i'd do anything 
to get you into my world 
and hold you within 
it's a right i defend 
overand overagain 
''')

['metal/rock']
